In [7]:
import fiona
from shapely.geometry import shape, mapping
from shapely.ops import transform
from shapely.geometry import Polygon
import copy

origShapeFile = 'E:/OSM/Schutzgebiete/Schongebiete-Alt/Schongebiete-4326.shp' # epsg:4326
newShapeFile = 'E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten-epsg4326.shp' # epsg:4326

shapesUpdateFile = 'E:/OSM/Schutzgebiete/New/new-shapes-dav.shp'
shapesIdenticalFile = 'E:/OSM/Schutzgebiete/New/same-shapes-dav.shp'
shapesDeletedFile = 'E:/OSM/Schutzgebiete/New/deleted-shapes-dav.shp'

In [13]:
# read and 3D to 2D convert DAV shapefile
def read_dav(shapeFile):
    newFeatures = []
    with fiona.open(shapeFile) as input:
        schema = input.schema
        crs = input.crs
        driver = input.driver
        for feat in input:
            if 'Kategorie' in feat['properties'] and feat['properties']['Kategorie'] != None and feat['properties']['Kategorie'] == "FFH-Gebiet (A)":
                print("Ignoring FFH-Gebiet (A)")
                continue
            if feat['geometry'] != None:
                if len(feat['geometry']['coordinates']) > 1:
                    # multipolygons - code to be improved...
                    for pfeat in feat['geometry']['coordinates']:
                        try: # some are len(1) lists, some aren't ?!
                            poly = Polygon(pfeat[0])
                        except:
                            poly = Polygon(pfeat)
                        poly = transform(lambda x, y, z=None: (x, y), poly)
                        feat2 = copy.deepcopy(feat)
                        feat2['geometry'] = mapping(poly)
                        newFeatures.append(feat2)
                    continue
                if len(feat['geometry']['coordinates'][0]) < 3:
                    print("Skipping 2-point line")
                    continue
                # transform 3D to 2D
                poly = shape(feat['geometry'])
                poly = transform(lambda x, y, z=None: (x, y), poly)
                feat['geometry'] = mapping(poly)
                newFeatures.append(feat)

    size = len(newFeatures);          
    print(f"Gebiete in {shapeFile}: {size}")
    return newFeatures, schema

In [14]:
oldFeatures, schema = read_dav(origShapeFile)
newFeatures, schema = read_dav(newShapeFile)

Gebiete in E:/OSM/Schutzgebiete/Schongebiete-Alt/Schongebiete-4326.shp: 235
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Ignoring FFH-Gebiet (A)
Gebiete in E:/OSM/Schutzgebiete/200126_Schutzgebiete_By-Karten/Schutzgebiete_BY-Karten-epsg4326.shp: 410


In [15]:
# iterate old shapes and check if they still exist in new shapes
# not working because it also writes signifcantly changed shapes
# -> writes shapesDeletedFile
delCount = 0 
foundCount = 0
missingFeaturesOut = []
sameFeatures = []

for oldFeature in oldFeatures:
    oldGeom = Polygon(shape(oldFeature['geometry']).exterior)
    oldGeomB = oldGeom.buffer(0.0001)
    found = False
    for newFeature in newFeatures:
        try:
            newGeom = Polygon(shape(newFeature['geometry']))
            newGeomB = newGeom.buffer(0.0001)
        except Exception as ex:
            continue
        try:
            iou = oldGeomB.intersection(newGeomB).area / oldGeomB.union(newGeomB).area
        except Exception as ex:
            print(ex)
            print(newGeom)
            print(oldGeom)
        if iou > 0.995:
            found = True
            foundCount +=1
            sameFeatures.append(oldFeature)
    if not found:
        delCount += 1
        missingFeaturesOut.append(oldFeature)
        print(f"Not found: {oldFeature['properties']}")

with fiona.open(shapesDeletedFile, 'w', crs={'init':'epsg:4326'}, driver='ESRI Shapefile', schema=schema) as out:
    for f in missingFeaturesOut:
        out.write(f)

print(f"Gefundene Gebiete = {foundCount}")
print(f"Gelöschte Gebiete = {delCount} -> in {shapesUpdateFile}")

Not found: OrderedDict([('Id', 0), ('Name', None), ('Regelung', 'Skitouren- und Schneeschuhgeher: Bitte nicht betreten oder befahren!')])
Not found: OrderedDict([('Id', 2), ('Name', None), ('Regelung', 'Skitouren- und Schneeschuhgeher: Bitte nicht betreten oder befahren!')])
Not found: OrderedDict([('Id', 3), ('Name', None), ('Regelung', 'Skitouren- und Schneeschuhgeher: Bitte nicht betreten oder befahren!')])
Not found: OrderedDict([('Id', 12), ('Name', None), ('Regelung', 'Skitouren- und Schneeschuhgeher: Bitte nicht betreten oder befahren!')])
Not found: OrderedDict([('Id', 18), ('Name', None), ('Regelung', 'Skitouren- und Schneeschuhgeher: Bitte nicht betreten oder befahren!')])
Not found: OrderedDict([('Id', 19), ('Name', None), ('Regelung', 'Skitouren- und Schneeschuhgeher: Bitte nicht betreten oder befahren!')])
Not found: OrderedDict([('Id', 20), ('Name', None), ('Regelung', 'Skitouren- und Schneeschuhgeher: Bitte nicht betreten oder befahren!')])
Not found: OrderedDict([('Id',

KeyboardInterrupt: 

In [22]:
newCount = 0 
foundCount = 0
newFeaturesOut = []
sameFeatures = []

for newFeature in newFeatures:
    newGeom = Polygon(shape(newFeature['geometry']).exterior)
    newGeomB = newGeom.buffer(0.0001)
    found = False
    for oldFeature in oldFeatures:
        try:
            oldGeom = Polygon(shape(oldFeature['geometry']))
            oldGeomB = oldGeom.buffer(0.0001)
        except Exception as ex:
            continue
        try:
            iou = newGeomB.intersection(oldGeomB).area / newGeomB.union(oldGeomB).area
        except Exception as ex:
            print(ex)
            print(newGeom)
            print(oldGeom)
        if iou > 0.995:
            found = True
            foundCount +=1
            sameFeatures.append(newFeature)
    if not found:
        newCount += 1
        newFeaturesOut.append(newFeature)

with fiona.open(shapesUpdateFile, 'w', crs={'init':'epsg:4326'}, driver='ESRI Shapefile', schema=schema) as out:
    for f in newFeaturesOut:
        out.write(f)

with fiona.open(shapesIdenticalFile, 'w', crs={'init':'epsg:4326'}, driver='ESRI Shapefile', schema=schema) as out:
    for f in sameFeatures:
        out.write(f)

print(f"Gefundene Gebiete = {foundCount}")
print(f"Neue/Geänderte Gebiete = {newCount} -> in {shapesUpdateFile}")

Gefundene Gebiete = 359
Neue/Geänderte Gebiete = 51 -> in E:/OSM/Schutzgebiete/New/new-shapes.shp
